<a href="https://colab.research.google.com/github/victorviro/Machine-Learning-Python/blob/master/Model_development_in_MLOps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model development

# Table of contents

1. [Introduction](#1)
2. [Establishing business objectives](#2)
3. [Data sources and EDA](#3)
4. [Feature engineering and feature selection](#4)
    1. [Feature stores](#4.1)
5. [Training](#5)
    1. [Stratified model training](#5.1)
6. [Evaluation](#6)
    1. [Model behavior](#6.1)
    2. [Specific test datasets](#6.2)
7. [Reproducibility](#7)
8. [Additional considerations](#8)
9. [References](#9)
    




# Introduction <a name="1"></a>

In previous notebooks, we [introduced MLOps](https://nbviewer.jupyter.org/github/victorviro/Machine-Learning-Python/blob/master/Introduction_to_MLOps.ipynb), and covered the [roles involved in the ML lifecycle](https://nbviewer.jupyter.org/github/victorviro/Machine-Learning-Python/blob/master/Roles_in_MLOps.ipynb). We also saw how we can [plan and scope ML projects](https://nbviewer.jupyter.org/github/victorviro/Machine-Learning-Python/blob/master/Planning_and_scoping_ML_projects.ipynb) in an efficient manner, and how to make a proper [ML experimentation](https://nbviewer.jupyter.org/github/victorviro/Machine-Learning-Python/blob/master/ML_experimentation.ipynb). In this notebook, we will take a deeper look into **ML model development**, which **can have an impact on MLOps after deployment**. For example, the next table shows why the development, specifically the choice of model, can ultimately affect MLOps. 



|Type  |Name | MLOps Considerations |
|:--- |:---- |:----:| 
|Linear| Linear Regression| There is a tendency for overfitting|
|Linear| Logistic Regression| -|
|Tree-Based| Decision Tree| Can be unstable—small changes in data can lead to a large change in the structure of the optimal decision tree.|
|Tree-Based| Random Forest| Predictions can be difficult to understand, which is challenging from a Responsible AI perspective. Random Forest models can also be relatively slow to output predictions, which can present challenges for applications.|
|Tree-Based| Gradient Boosting| Like Random Forest, predictions can be difficult to understand. Also, a small change in the feature or training set can create radical changes in the model.|
|Deep Learning| Neural Networks| In terms of interpretability, deep learning models are almost impossible to understand. Deep learning algorithms, including neural networks, are also extremely slow to train and require a lot of computational power (and data). Is it worth the resources, or would a simpler model work just as well?|



# Establishing business objectives <a name="2"></a>



The process of **developing a ML model** typically **starts with a business objective** (e.g reducing fraudulent transactions to < 0.1%). 
In the notebook [Planning and scoping ML projects](https://nbviewer.jupyter.org/github/victorviro/Machine-Learning-Python/blob/master/Planning_and_scoping_ML_projects.ipynb), we discussed why this part is **one of the main reasons that so many ML projects end up failing**, what questions we need to ask to the business to fully understand what they need and expect, and what requirements and assumptions they have. Business objectives naturally come with **performance targets**, technical **infrastructure requirements**, and cost constraints; all of these factors can be captured as key performance indicators (**KPIs**), which will ultimately **enable the business performance** of models in production to be monitored. 

ML projects are generally part of a larger project, which may provide some guidance for how the ML model should be built. For example, the required degree of transparency will strongly influence the choice of algorithms and may drive the need to provide explanations together with predictions.

# Data sources and EDA <a name="3"></a>

With business objectives defined, the next step is the **search for suitable input data**. In the notebook [ML experimentation](https://nbviewer.jupyter.org/github/victorviro/Machine-Learning-Python/blob/master/ML_experimentation.ipynb), we listed out some key questions we need to ask for finding data to build ML models.

In the notebook [ML experimentation](https://nbviewer.jupyter.org/github/victorviro/Machine-Learning-Python/blob/master/ML_experimentation.ipynb), we also discussed that,
since data is essential to power ML models, it always helps to **understand the patterns in data before attempting to train models**. ***Exploratory data analysis (EDA)*** techniques can **help build hypotheses about the data, identify data cleansing requirements, incompleteness, inconsistency, and inform the process of selecting potentially significant features**. EDA can be carried out visually and statistically if more rigor is required. 

An example of such a process can include:

- Documenting **how the data was collected** and what assumptions were already made.

- Summarizing statistics of the data: what is the type of each column? Are there **missing values** or **outliers**? What is the **distribution of the data**?.

- **Cleaning**, filling, reshaping, filtering, clipping, sampling, etc.

- Checking **correlations** between the different columns, running **statistical tests** on some subpopulations, fit distribution curves.

# Feature engineering and feature selection  <a name="4"></a>

**Feature Engineering** is the process of **taking raw data from the dataset and transforming it into "features" that better represent the underlying problem to be solved**. This step is intensive in terms of **data handling**, but it is important since **feature engineering can make significant improvements** to the algorithms. It also reduces modeling debt, allowing data scientists to understand the main prediction features. Of course, there are **trade-offs** to consider between **the cost of time spent** to "understand" the model **and the expected value**. 

Feature engineering techniques can be divided into four types:

- *Derivatives*: Infer new information from existing information (e.g., what day of the week is this date?).

- *Enrichment*: Add new external information (e.g., is this day a public holiday?).

- *Encoding*: Present the same information differently.  
 - **ML algorithms require numbers as input**. When an input variable is categorical, we may need to transform it (e.g. one-hot encoding). When the input data is not tabular, we need to transform it (e.g. embeddings for text or images).
 - Datasets can contain **features with different scales**, and it can have an adverse effect for certain ML models, especially when minimizing the cost function of the algorithm. There are common ways to get all attributes to have the same scale (e.g. min-max scaling, normalization).

 As shown in this [notebook](https://nbviewer.jupyter.org/github/victorviro/Machine-Learning-Python/blob/master/Introduction_to_Scikit_learn.ipynb#4), the Scikit-learn module provides built-in methods for preprocessing tabular data.

- *Combination*: Link features together (e.g., interactions between multiple variables to create new variables).

The question here is how much and when to stop. **Adding more features may produce a more accurate model**, or compensate for some useful missing information. However, it also comes with **downsides**. The model may suffer **overfitting**, it can become more **expensive to compute**, it may require **more maintenance**, and it can lose some stability.

Features vary in significance. one can look at the **correlation with the target variable** or quickly train a simple model and look at which features are the strongest predictors. **Automated feature selection** can help by estimating how important the features will be for the predictive performance of the model. Tools such as **[auto-sklearn](https://automl.github.io/auto-sklearn/master/) or AutoML** that cross-reference features against a given target to estimate which features are likely to yield the best results.

## Feature stores <a name="4.1"></a>

Feature factories/stores are **repositories of different features associated with business entities** that are created and **stored in a central location for easier reuse**. Given how time-consuming feature engineering is for data scientists, this concept has huge potential to free up their time for even more valuable tasks. For example, [Feast](https://feast.dev/) is an open-source feature store.

# Training <a name="5"></a>



The process of training and optimizing an ML model is **iterative**; several **algorithms may be tested**, features can be generated, **feature selections may be adapted**, and algorithm **hyper parameters tuned**. Because of its iterative nature, training is also the most **intensive step** in terms of **computing power**. Moreover, **trying all combinations** of every possible hyperparameter, feature handling, etc., quickly becomes **unmanagable**. Therefore, it is useful to define a time or computation budget for experiments as well as an acceptability threshold for the usefulness of the model. Once again, tools like auto-sklearn or AutoML can help to automate this process. Different approaches for hyperparameter tuning are shown in the notebook [Hyperparameter optimization](https://nbviewer.jupyter.org/github/victorviro/Machine-Learning-Python/blob/master/Hyperparameter_Optimization.ipynb).

Some ML algorithms can best support specific use cases, but **governance considerations** may also **play a part in the choice of algorithm**. In particular, highly regulated environments where decisions must be explained (e.g., financial services) cannot use opaque algorithms, like neural networks, and have to favor simpler techniques, such as decision trees. In many use cases, it’s not so much a trade-off on performance but rather a **trade-off on cost**. That is, **simpler techniques usually require more costly manual feature engineering to reach the same level of performance as more complex techniques**.

## Stratified model training <a name="5.1"></a>

Some tools offer **stratified model training**. For example, say the business wants to predict customer demand for products to optimize inventory, but behavior varies a lot from one store to the next. Stratified modeling consists of training one model per store that can be better optimized for each store rather than a model that tries to predict in all stores.

# Model evaluation <a name="6"></a>

> ”Essentially, all models are wrong, but some are useful” - statistician George Box

It’s important to evaluate a model in context and **compare it to what existed before** to get an idea of the outcome of replacing the current model with the new one. 

- A model with a performance that could be considered disappointing can still possibly enhance an existing situation. For instance, having a slightly more accurate forecast of demand for a certain product may have huge cost-saving impacts. 
- On the contrary, a model that gets a **perfect score is suspicious**, as most problems have noise in the data that’s hard to predict. It may be a sign that there is a leak in the data (the target to be predicted is also in the input data or an input feature is very correlated to the target but, practically, available only once the target is known), or that the model overfits the training data and will not generalize well.

Choosing the proper metric is important. For example, accuracy is often used for classification problems but is not adequate when the classes are unbalanced (the reason of this can be found in this [notebook](https://nbviewer.jupyter.org/github/victorviro/Machine-Learning-Python/blob/master/ML_metrics_and_cost_functions.ipynb#Accuracy-)). The metric chosen must match the problem at hand, which means **understanding the limits and tradeoffs of the metric** and their impact. Calculating additional metrics is usually cheap and fast, so we can compute all of the applicable ones and recording them. It can help if, during the process of developing a model and tuning it, we decide to use a different metric.

The metrics should be **evaluated on the holdout/test set**. Instead of just a simple split, [**cross-validation k-fold**](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) allows us to rotate the parts that we hold out to evaluate and train multiple times, which gives an idea of the stability of the model. 

With a simple split, the holdout set can consist of the most recent records instead of the randomly chosen ones. This likely leads to more realistic estimations of how well the model will generalize. Moreover, it allows to assess whether the data drifted between the training and the holdout dataset.

## Model behavior <a name="6.1"></a>

Beyond the raw metrics, when evaluating a model, it can be critical to **understand how the model will behave**. For example, data scientists should ensure that the model is not actively harmful (a model that would predict that all patients need to be checked by a doctor may score high in terms of raw prevention, but not so much on realistic resource allocation).

Examples of these reasonable steps include:

- Cross-checking **different metrics**.

- Model stability: When changing one feature slightly, one expects small changes in the outcome. Checking how the model reacts to different inputs (e.g., plot the average prediction for different values of some inputs and see whether there is **extreme variability**). Generally, simpler models or more regularized ones show better stability.

- Splitting data into **subpopulations** based on a "sensitive" variable (that may or may not be used as a feature of the model) and check the differences across them (e.g., is the error rate the same for males and females?). This is how fairness is evaluated. Models that apply to people usually have to be analyzed for fairness (it can have regulatory and reputational implications).

## Specific test datasets <a name="6.2"></a>

There are some scenarios in which the test data should not always match "real-world" data. For example, it can be useful to prepare a certain number of scenarios, and while some of them should match realistic situations, other **data** should be **specifically generated in ways that could be problematic (e.g., extreme values, missing values)**.

# Reproducibility <a name="7"></a>

**Keeping track of each experiment** is important so data scientists can recreate the best results. An **experiment tracking tool** can simplify the process of remembering the data, the features selected, and model parameters alongside the performance metrics. These enable experiments to be compared side-by-side, highlighting the differences in performance.

Versions of a model usually need to be saved for possible later use. The challenge here is **reproducibility**. That is, **save enough information about the environment where the model was developed** so that **the model can be reproduced with the same results** again from scratch.

Without reproducibility, data scientists will not be able to confidently iterate on models, and worse, they are unlikely to be able to hand over the model to DevOps to see if what was created in the lab can be faithfully reproduced in production. True reproducibility requires **version control of all** of the assets and **parameters involved, including the data and code used to train and evaluate the model, the metrics, as well as a record of the software environment** (for instance, a slightly different version of a Python package involved in one step may change the results in ways that can be hard to predict).

An experiment tracking tool (like [MLflow](https://www.mlflow.org/docs/latest/tracking.html) or [Weights & Biases](https://wandb.ai/site)) can help to log information like hyperparameters or metrics. A containerization service (like [Docker](https://www.docker.com/)) allows us to package the software dependencies and libraries needed to run our code and it introduces **portability** to the ML project. To version the code, the most common version control system is [***Git***](https://git-scm.com/).

# Additional considerations <a name="8"></a>

- The **DevOps** team also **needs to understand how to verify the model** (i.e., what does the model do, **how should it be tested, and what are the expected results?**). **Documentation** is still the standard solution.

- ML models are challenging to understand. While model algorithms come with standard **performance measures** to assess their efficacy, these **don’t explain how the predictions are made**. The "how" is important to sanity-check the model or **help better engineer features**, and it may be necessary to ensure that **fairness requirements**. This is the field of **explainability/interpretability**, which is connected to Responsible AI as discussed in the Introduction to MLOps notebook. Explainability techniques are becoming increasingly important as global concerns grow about the impact of AI. They offer a way to mitigate uncertainty and help prevent unintended consequences. A review of the **techniques** most commonly use today is available in the notebook [ML interpretability](https://nbviewer.jupyter.org/github/victorviro/Machine-Learning-Python/blob/master/Machine_Learning_Interpretability_with_InterpretML.ipynb).

# References <a name="9"></a>

- [MLOps: Continuous delivery and automation pipelines in machine learning](https://cloud.google.com/solutions/machine-learning/mlops-continuous-delivery-and-automation-pipelines-in-machine-learning)

- [Introducing MLOps](https://www.oreilly.com/library/view/introducing-mlops/9781492083283/)

- [MLinproduction blog](https://mlinproduction.com/)

- [MLOps.Community](https://www.youtube.com/channel/UCG6qpjVnBTTT8wLGBygANOQ)